In [0]:
import dlt
from pyspark.sql.functions import *
query = spark.sql('''with isMailing0 as
(select ismailing as isMailing0, agencyid as agencyid0, State as State0 from dev.bronze_dbmars_dbo.tb_am_AgencyAddresses where ismailing =0),

isMailing1 as
(select ismailing as isMailing1, agencyid as agencyid1, State as State1 from dev.bronze_dbmars_dbo.tb_am_AgencyAddresses where ismailing=1),

joinIsMailing as
(select * from ismailing0 full outer join ismailing1 on ismailing0.agencyid0=ismailing1.agencyid1)


SELECT
--Fact Data
trim(fpol.PolicyNumber) as PolicyNumber
,trim(whtasks.WorkflowHistoryTaskId) as workflowHistoryTaskID
,trim(whtasks.WorkflowHistoryId) as workflowhistoryID
,trim(f.FileId) as fileID
,trim(doc.DocumentId) as documentID
,trim(fpol.DepartmentId) as fpolDepartmentID
--,trim(cu.DepartmentID) as cuDepartmentID
--,trim(uu.DepartmentID) as uuDepartmentID
,trim(fpol.CompanyID) as companyID
,trim(fpol.EffectiveDate) as effectiveDate
,trim(fpol.ExpirationDate) as expirationDate
,trim(whtasks.Initiated) as initiated
,trim(wsteps.WorkflowID) as workflowID
,trim(wsteps.WorkflowStepID) as workflowStepID
,trim(whtasks.DueDate) as dueDate
,trim(whtasks.DueDateDate) as dueDateDate
,trim(f.AgencyContactId) as agencyContactID
,trim(sr.RegionId) as srRegionID
,trim(r.RegionID) as fileRegionID
--,trim(whtasks.CompletedDate) as completedDate
,trim(whtasks.Completed) as completed
,trim(f.AgencyId) as agencyID
,trim(whtasks.AssignedToUserId) as assignedToUserID
,trim(whtasks.CompletedByUserId) as completedByUserID
,trim(wsteps.ExcludeFromMaster) as excludedFromMaster
,trim(whtasks.LastModified) as lastModified
--,trim(doc.Deleted) as docDeleted
--,trim(fpol.Deleted) as fpolDeleted
--,trim(f.Deleted) as fDeleted
--Dim Data
,trim(f.FileNumber) as fileNumber
,trim(wsteps.DepartmentType) as departmentType
,trim(c.CompanyName) as companyName
,trim(w.WorkflowName) as workflowName
,trim(wsteps.StepName) as stepName
,trim(r.RegionName) as regionName --look into how this works when the fregion is different from the sr region
,trim(f.FileName) as fileName
,trim(whtasks.TaskNote) as taskNote
,trim(whtasks.CompletedStatus) completedStatus
,trim(a.AgencyName) as agencyName
,concat(trim(uu.firstname),' ',trim(uu.lastname)) as assignedToUserName
,concat(trim(cu.firstname),' ',trim(cu.lastname))  as completedByUserName
,COALESCE(trim(whtasks.Locked), '0') as locked
,trim(whtasks.CompletedType) as completedType
,trim(ss.State) as agencyState 
--calculations
,DATEDIFF(day, whtasks.Initiated,whtasks.Completed) AS DaysTakenToComplete
,Case when whtasks.Completed is null AND DueDate > '2023-01-01 00:00:00.000'
				AND coalesce(doc.Deleted, fpol.Deleted, f.Deleted, cast(0 as boolean))=0
			then 'Pending'
			when whtasks.Completed is NOT null 
				AND whtasks.Completed > '2024-01-01 00:00:00.000' 
				AND whtasks.DueDate > '2023-01-01 00:00:00.000'
				AND coalesce(doc.Deleted, fpol.Deleted, f.Deleted, cast(0 as boolean))=0
				AND  CASE
    				WHEN whtasks.CompletedStatus = 'Task Updated' OR COALESCE(whtasks.CompletedType, '') != 'terminated' THEN 1
    				ELSE 0
  					END = 1
			then 'Completed'
	end as CompletedOrPending
,CASE 
		WHEN whtasks.Completed IS NULL AND whtasks.DueDate < current_timestamp() THEN 'OSS' 
		WHEN whtasks.Completed IS NULL AND whtasks.DueDate > current_timestamp() THEN 'Active' 
		ELSE NULL
		END AS StandardStatus

,CASE WHEN wsteps.WorkflowStepID = 60 THEN coalesce(uu.DepartmentID,cu.DepartmentID, fpol.DepartmentID)
	        ELSE CASE
			 when whtasks.completed IS NOT NULL and
                    case when wsteps.DepartmentType = 'assigneddepartment' THEN fpol.DepartmentID
                    when wsteps.DepartmentType != 'assigneddepartment' then coalesce(wsteps.ResponsibleDepartmentId,fpol.DepartmentID) 
                    End is not null
                then
                    case when wsteps.DepartmentType = 'assigneddepartment' THEN fpol.DepartmentID
                    when wsteps.DepartmentType != 'assigneddepartment' then coalesce(wsteps.ResponsibleDepartmentId,fpol.DepartmentID) 
                    End
                end 
end as ResponsibleDepartmentId

 

FROM dev.bronze_dbmars_dbo.tb_dm_workflowhistorytasks as whtasks
INNER JOIN dev.bronze_dbmars_dbo.tb_dm_WorkflowHistory as wh on wh.WorkflowHistoryId = whtasks.WorkflowHistoryId
INNER JOIN dev.bronze_dbmars_dbo.tb_dm_WorkflowSteps as wsteps on wsteps.WorkflowStepID = whtasks.WorkflowStepId
LEFT JOIN dev.bronze_dbmars_dbo.tb_shared_users as uu on uu.UserId = whtasks.AssignedToUserId
LEFT JOIN dev.bronze_dbmars_dbo.tb_shared_users cu on cu.UserId = whtasks.CompletedByUserId
INNER JOIN dev.bronze_dbmars_dbo.tb_dm_Document as doc on doc.DocumentId = wh.DocumentId 
LEFT JOIN dev.bronze_dbmars_dbo.tb_dm_File as f on f.FileId = doc.FileId
LEFT JOIN dev.bronze_dbmars_dbo.tb_dm_FilePolicies as fpol ON fpol.FilePolicyId = doc.FilePolicyId
INNER JOIN dev.bronze_dbmars_dbo.tb_dm_Workflows as w ON w.WorkflowID = wsteps.WorkflowId
LEFT JOIN dev.bronze_dbmars_dbo.tb_am_Agencies as a ON a.AgencyId = f.AgencyId
LEFT JOIN joinIsMailing jim on jim.agencyid0= a.agencyid
LEFT JOIN dev.bronze_dbmars_dbo.tb_shared_States AS ss ON ss.State = 
                    case when coalesce(isMailing0, isMailing1)=0 then State0 else State1 end
LEFT JOIN dev.bronze_dbmars_dbo.tb_shared_Regions r ON r.RegionId = f.RegionId
LEFT JOIN dev.bronze_dbmars_dbo.tb_shared_Regions AS sr ON sr.RegionId = ss.RegionId 
LEFT JOIN dev.bronze_dbmars_dbo.tb_shared_Companies as c ON c.CompanyId = fpol.CompanyId
''')

@dlt.table(
  comment="silver workflow"
)
def Workflow():
    return query